In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os,sys
import errno
import string
from nltk.corpus import reuters 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score,accuracy_score,roc_auc_score,mean_absolute_error
from sklearn.preprocessing import label_binarize
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
import csv
from sklearn import svm
import warnings
import random
from statistics import mode
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
random.seed(0)
warnings.filterwarnings('ignore')
def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)
def Cosine(a, b):#distance
    return distance.cosine(a,b)
def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return 1-isc
def most_common(lst):
    return(mode(lst))


def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["k",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=train_time+test_time
    for i in range(0,len(Arr)):
        print(i,len(Arr))
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    averageMeanPrecison=[]
    kList=[5,15,30,45]
    func=globals()[metric]
    train_time=[]
    test_time=[]
    ##########Main piece of code############
    # split training data according to class distribution
    for k in kList:
        time1=time.time()
        pred=[]
        print("k",k)
        k1=k
        if k1<15:
            k1=k*(len(classes)+1) 
        classifier = KNeighborsClassifier(n_neighbors=k1,metric=func)
        if len(train_labels)<k1:
            k=len(train_labels)
            classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        classifier.fit(train_data,train_labels)
        distances,neighs=classifier.kneighbors(test_data,return_distance=True)
        time2=time.time()
        for t in range(0,test_data.shape[0]):
            dist=distances[t,:]
            neighs2=neighs[t,:]
            weights2=np.ones((len(dist)))
            if (dist[-1]-dist[0]) !=0:
                weights2 = (dist[-1] - dist)/(dist[-1]-dist[0])
            neigh_labels=[train_labels[n] for n in neighs2]
            maxweight=-np.inf
            for lbl in set(neigh_labels):
                weightSum=np.nansum([weights2[j]/(distances[t,j]+0.00001) for j,ii in enumerate(neighs[t]) if train_labels[ii]==lbl])
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            pred.append(predLabel)
        y_pred.append(pred) 
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
              
    class_dict={}
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)

        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)

    #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Datasets April\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Datasets April\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    #pca=PCA(n_components=3)
    #arr=pca.fit_transform(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = preprocessing.LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    classes=list(set(labels))
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "EDWKNN\\April Task1\\Cancelled Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_EDWKNN.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=xTrain
            test_data=xTest
            train_labels=yTrain
            test_labels=yTest
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





[nltk_data] Downloading package reuters to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


b'Datasets April\\1_lung-cancer'
###############
k 5
k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+-----------------------+-----------------------+-----------------------+
|    k     | Accuracy |       Train Time      |       Test time       |       Total Time      |
+----------+----------+-----------------------+-----------------------+-----------------------+
|    5     |   0.75   |     0:00:00.002987    |        0:00:00        |     0:00:00.002987    |
|    15    |   0.75   |     0:00:00.000970    |        0:00:00        |     0:00:00.000970    |
|    30    |   0.75   |     0:00:00.000997    |        0:00:00        |     0:00:00.000997    |
|    45    |   0.75   |     0:00:00.000997    |     0:00:00.000998    |     0:00:00.000997    |
| Average  |   0.75   |     0:00:00.001488    |     0:00:00.000249    |     0:00:00.000869    |
| Std Dev. |   0.0    | 0.0008657578244265996 | 0.0004319489347260107 | 0.0009228115456086615 |
+----

metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+------------------------+-----------------------+-----------------------+
|    k     | Accuracy |       Train Time       |       Test time       |       Total Time      |
+----------+----------+------------------------+-----------------------+-----------------------+
|    5     |   1.0    |     0:00:00.001995     |        0:00:00        |     0:00:00.001995    |
|    15    |   1.0    |     0:00:00.000997     |        0:00:00        |     0:00:00.000997    |
|    30    |  0.6667  |     0:00:00.000997     |        0:00:00        |     0:00:00.000997    |
|    45    |  0.6667  |     0:00:00.000998     |     0:00:00.000997    |     0:00:00.000998    |
| Average  |  0.8333  |     0:00:00.001247     |     0:00:00.000249    |     0:00:00.000748    |
| Std Dev. |  0.1667  | 0.00043205220589065223 | 0.0004316392199066565 | 0.0006597453589602204 |
+----------+----------+------------------------+-----------------------+-----

+----------+----------+-----------------------+-----------------------+------------------------+
|    k     | Accuracy |       Train Time      |       Test time       |       Total Time       |
+----------+----------+-----------------------+-----------------------+------------------------+
|    5     |   1.0    |     0:00:00.000987    |     0:00:00.000998    |     0:00:00.000987     |
|    15    |   1.0    |     0:00:00.000997    |        0:00:00        |     0:00:00.000997     |
|    30    |  0.6667  |     0:00:00.000997    |        0:00:00        |     0:00:00.000997     |
|    45    |  0.6667  |     0:00:00.000997    |        0:00:00        |     0:00:00.000997     |
| Average  |  0.8333  |     0:00:00.000995    |     0:00:00.000249    |     0:00:00.000622     |
| Std Dev. |  0.1667  | 4.645722290313213e-06 | 0.0004319489347260107 | 0.00048181452025286136 |
+----------+----------+-----------------------+-----------------------+------------------------+
metric Euclidean
measure Preci

k 15
k 30
k 45
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+-----------------------+----------------------+
|    k     | Accuracy |      Train Time      |       Test time       |      Total Time      |
+----------+----------+----------------------+-----------------------+----------------------+
|    5     |  0.5652  |    0:00:00.126648    |     0:00:00.009974    |    0:00:00.126648    |
|    15    |  0.587   |    0:00:00.117685    |     0:00:00.002992    |    0:00:00.117685    |
|    30    |  0.5652  |    0:00:00.116688    |     0:00:00.005985    |    0:00:00.116688    |
|    45    |  0.5652  |    0:00:00.123669    |     0:00:00.006981    |    0:00:00.123669    |
| Average  |  0.5707  |    0:00:00.121173    |     0:00:00.006483    |    0:00:00.063828    |
| Std Dev. |  0.0094  | 0.004138072996067981 | 0.0024934768869391195 | 0.057446521251028344 |
+---

...

k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+---------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time      |      Total Time     |
+----------+----------+----------------------+---------------------+---------------------+
|    5     |   1.0    |    0:00:00.140592    |       0:00:00       |    0:00:00.140592   |
|    15    |   1.0    |    0:00:00.171836    |    0:00:00.015620   |    0:00:00.171836   |
|    30    |   1.0    |    0:00:00.140592    |    0:00:00.015622   |    0:00:00.140592   |
|    45    |   1.0    |    0:00:00.156214    |       0:00:00       |    0:00:00.156214   |
| Average  |   1.0    |    0:00:00.152309    |    0:00:00.007811   |    0:00:00.080060   |
| Std Dev. |   0.0    | 0.012953103497326982 | 0.00781059266774209 | 0.07303640019090814 |
+----------+----------+----------------------+---------------------+---------------------+
metric Euclidean


k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+-----------------------+-----------+--------------------+
|    k     | Accuracy |       Train Time      | Test time |     Total Time     |
+----------+----------+-----------------------+-----------+--------------------+
|    5     |  0.9273  |     0:00:00.171834    |  0:00:00  |   0:00:00.171834   |
|    15    |  0.9273  |     0:00:00.171833    |  0:00:00  |   0:00:00.171833   |
|    30    |  0.9273  |     0:00:00.171837    |  0:00:00  |   0:00:00.171837   |
|    45    |  0.9273  |     0:00:00.171836    |  0:00:00  |   0:00:00.171836   |
| Average  |  0.9273  |     0:00:00.171835    |  0:00:00  |   0:00:00.085917   |
| Std Dev. |   0.0    | 1.594911594421822e-06 |    0.0    | 0.0859174132421124 |
+----------+----------+-----------------------+-----------+--------------------+
metric Euclidean
measure Precision
0 4
1 4
2 4
3 4
+----------+-----------+-----------------------+-----------+------

k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time     |
+----------+----------+----------------------+----------------------+---------------------+
|    5     |   1.0    |    0:00:00.140591    |    0:00:00.015622    |    0:00:00.140591   |
|    15    |   1.0    |    0:00:00.156213    |       0:00:00        |    0:00:00.156213   |
|    30    |   1.0    |    0:00:00.156213    |       0:00:00        |    0:00:00.156213   |
|    45    |   1.0    |    0:00:00.171835    |       0:00:00        |    0:00:00.171835   |
| Average  |   1.0    |    0:00:00.156213    |    0:00:00.003905    |    0:00:00.080059   |
| Std Dev. |   0.0    | 0.011046267526381028 | 0.006764378131242274 | 0.07670264966398507 |
+----------+----------+----------------------+----------------------+---------------------+
metric 

k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+-----------+--------------------+
|    k     | Accuracy |      Train Time      | Test time |     Total Time     |
+----------+----------+----------------------+-----------+--------------------+
|    5     |  0.9273  |    0:00:00.140592    |  0:00:00  |   0:00:00.140592   |
|    15    |  0.9273  |    0:00:00.156213    |  0:00:00  |   0:00:00.156213   |
|    30    |  0.9273  |    0:00:00.156213    |  0:00:00  |   0:00:00.156213   |
|    45    |  0.9636  |    0:00:00.156214    |  0:00:00  |   0:00:00.156214   |
| Average  |  0.9364  |    0:00:00.152308    |  0:00:00  |   0:00:00.076154   |
| Std Dev. |  0.0157  | 0.006764481375817807 |    0.0    | 0.0763040915384203 |
+----------+----------+----------------------+-----------+--------------------+
metric Euclidean
measure Precision
0 4
1 4
2 4
3 4
+----------+-----------+----------------------+-----------+-----------------

k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time     |
+----------+----------+----------------------+----------------------+---------------------+
|    5     |   1.0    |    0:00:00.156211    |    0:00:00.015621    |    0:00:00.156211   |
|    15    |   1.0    |    0:00:00.140596    |       0:00:00        |    0:00:00.140596   |
|    30    |   1.0    |    0:00:00.171832    |       0:00:00        |    0:00:00.171832   |
|    45    |   1.0    |    0:00:00.187456    |       0:00:00        |    0:00:00.187456   |
| Average  |   1.0    |    0:00:00.164024    |    0:00:00.003905    |    0:00:00.083965   |
| Std Dev. |   0.0    | 0.017463576836794937 | 0.006764171654696038 | 0.08114712053739578 |
+----------+----------+----------------------+----------------------+---------------------+
metric 

In [2]:
"test"

'test'